In [1]:
using Distributed
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
@everywhere using DelimitedFiles
@everywhere using Revise, MDToolbox
ENV["COLUMNS"]=110; #display width for MDToolbox

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /Users/yasu/.julia/packages/Revise/moD4B/src/packagedef.jl:1361


In [3]:
receptor=readpdb("1cgi_receptor.pdb")
ligand=readpdb("1cgi_ligand.pdb")

1x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|    64.86    34.03    29.79 |    65.13    34.96    28.68 |  …      51.32    20.79    29.47 |

In [4]:
receptor =  MDToolbox.set_radius(receptor)
ligand = MDToolbox.set_radius(ligand)

1x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|    64.86    34.03    29.79 |    65.13    34.96    28.68 |  …      51.32    20.79    29.47 |

In [5]:
receptor_sasa = MDToolbox.compute_sasa(receptor, 1.4)
ligand_sasa = MDToolbox.compute_sasa(ligand, 1.4)

440-element Array{Float64,1}:
 33.14939845728653
 12.831049795424109
  4.528605810149686
 32.81343261192081
 25.158921167498253
  9.183006226136861
 35.71530080209068
 52.34523773806415
  4.898364720492511
  7.296087138574493
  4.528605810149686
 30.13478505176401
 20.378726145673582
  ⋮
  0.0
  5.357295120313601
 16.353298758873866
 26.416867225873165
 23.0204128682609
  0.0
  0.25158921167498255
  2.6416867225873166
 31.027667571816274
 15.346941912173934
  0.26808257310632905
 45.53700852266561

In [6]:
receptor = TrjArray(receptor, charge=receptor_sasa)
writepdb("test.pdb", receptor)

In [7]:
receptor = TrjArray(receptor, mass=receptor_sasa)
ligand = TrjArray(ligand, mass=ligand_sasa)

1x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|    64.86    34.03    29.79 |    65.13    34.96    28.68 |  …      51.32    20.79    29.47 |

In [8]:
quaternions = readdlm("../data/quaternion/QUATERNION_LIST_576_Orient");
# quaternions = quaternions[1:10, :]
size(quaternions)
quaternions = quaternions[1:50, :]

50×4 Array{Float64,2}:
  0.809511  0.106574   0.351469    0.458043
  0.754344  0.31246    0.220942    0.533402
  0.64777   0.497052   0.0753591   0.572411
  0.497052  0.64777   -0.0753591   0.572411
  0.31246   0.754344  -0.220942    0.533402
  0.106574  0.809511  -0.351469    0.458043
 -0.106574  0.809511  -0.458043    0.351469
 -0.31246   0.754344  -0.533402    0.220942
 -0.497052  0.64777   -0.572411    0.0753591
 -0.64777   0.497052  -0.572411   -0.0753591
 -0.754344  0.31246   -0.533402   -0.220942
 -0.809511  0.106574  -0.458043   -0.351469
  0.905061  0.119154   0.105662    0.394338
  ⋮                               
  0.776649  0.595944   0.0266441   0.202378
  0.595944  0.776649  -0.0266441   0.202378
  0.374626  0.904427  -0.0781151   0.188586
  0.127778  0.97057   -0.124263    0.161943
 -0.127778  0.97057   -0.161943    0.124263
 -0.374626  0.904427  -0.188586    0.0781151
 -0.595944  0.776649  -0.202378    0.0266441
 -0.776649  0.595944  -0.202378   -0.0266441
 -0.904427  0

In [9]:
grid_space = 1.2

1.2

In [10]:
@time result = MDToolbox.dock_fft(receptor, ligand, quaternions, grid_space=grid_space, tops=10)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:32


124.570605 seconds (32.70 M allocations: 2.954 GiB, 2.83% gc time)


(receptor = 1x1799 TrjArray{Float64,Int64}
| E                          | E                          |  …   E                          |
| 1CYS                       | 1CYS                       |  …   245ASN                     |
| 1N                         | 2CA                        |  …   1799OXT                    |
|   -12.82    -4.83   -11.13 |   -12.18    -4.38    -9.89 |  …      -0.75     7.40   -21.12 |, ligand = 10x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|   -22.61   -27.05   -10.17 |   -24.03   -26.72    -9.99 |  …      -8.36   -15.83    -4.70 |
|   -22.61   -25.85   -10.17 |   -24.03   -25.52    -9.99 |         -8.36   -14.63    -4.70 |
|   -22.61   -25.85    -8.97 |   -24.03   -25.52    -8.79 |         -8

In [11]:
result.score

10-element Array{Float64,1}:
 3.25899523939176e-5
 3.25899523939176e-5
 3.25899523939176e-5
 3.258995239391759e-5
 3.258995239391759e-5
 3.258995239391759e-5
 3.258995239391759e-5
 3.258995239391759e-5
 3.258995239391759e-5
 3.2589952393917585e-5

In [12]:
result.ligand

10x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|   -22.61   -27.05   -10.17 |   -24.03   -26.72    -9.99 |  …      -8.36   -15.83    -4.70 |
|   -22.61   -25.85   -10.17 |   -24.03   -25.52    -9.99 |         -8.36   -14.63    -4.70 |
|   -22.61   -25.85    -8.97 |   -24.03   -25.52    -8.79 |         -8.36   -14.63    -3.50 |
|   -22.61   -31.85   -10.17 |   -24.03   -31.52    -9.99 |         -8.36   -20.63    -4.70 |
|   -22.61   -30.65   -10.17 |   -24.03   -30.32    -9.99 |         -8.36   -19.43    -4.70 |
|   -22.61   -29.45   -10.17 |   -24.03   -29.12    -9.99 |  …      -8.36   -18.23    -4.70 |
|   -22.61   -28.25   -10.17 |   -24.03   -27.92    -9.99 |         -8.36   -17.03    -4.70 |
|   -22.61   -28.25    -8.97 

In [13]:
writepdb("result_receptor.pdb", result.receptor)
writepdb("result_ligand.pdb", result.ligand)